In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

In [ ]:
import torch
from vllm import LLM, SamplingParams
from vllm.attention.backends.abstract import AttentionType, SharedSelfAttentionType

/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-11 12:08:46,479	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we 

### Tests

In [ ]:
# enforce xformers attention backend - changes are made here
os.environ["VLLM_ATTENTION_BACKEND"] = "XFORMERS_CLA"

```python
# Inside LlamaAttention.forward
# Change q to fixed input for testing.
if self.cache_config.debug_kv_sharing: q = torch.ones_like(q)
attn_output = self.attn(q, k, v, kv_cache, attn_metadata, compute_new_kv=compute_new_kv_map)
if self.cache_config.debug_kv_sharing: self.attn_outputs.append(attn_output)
```

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    "Reset attn outputs and attn metadatas from warmup."
    for layer in model.model.layers:
        if len(layer.self_attn.attn_outputs) > 0:
            layer.self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

### TEST 1

In [ ]:
### test 1: No prefix caching, no quantization, use v1 block manager, no cuda graphs ###
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = True
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16", 
                         max_model_len=1024*1, 
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

WARNING 10-10 15:31:59 config.py:365] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-10 15:31:59 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forw

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-10 15:32:00 selector.py:122] Using XFormers CLA backend.


/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 10-10 15:32:01 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...
INFO 10-10 15:32:01 selector.py:122] Using XFormers CLA backend.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.33it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.32it/s]



INFO 10-10 15:32:02 model_runner.py:1060] Loading model weights took 2.8008 GB
INFO 10-10 15:32:03 gpu_executor.py:122] # GPU blocks: 5937, # CPU blocks: 65536


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 51.28it/s, est. speed input: 154.39 toks/s, output: 51.45 toks/s]


In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.equal(attn_output_layer0, attn_output_layer1)

# 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
assert len(model.model.attn_metadatas) == 3 
# Prefill phase.
attn_metadata = model.model.attn_metadatas[0]
assert attn_metadata.decode_metadata is None
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# Decode phase 1.
attn_metadata = model.model.attn_metadatas[1]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# Decode phase 2.
attn_metadata = model.model.attn_metadatas[2]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 20.99it/s, est. speed input: 63.06 toks/s, output: 63.05 toks/s]


### TEST 2 (fp8)

```python
if kv_cache_dtype in ("fp8", "fp8_e4m3"):
    target_dtype = torch.float8_e4m3fn
elif kv_cache_dtype == "fp8_e5m2":
    target_dtype = torch.float8_e5m2
```

Quantiztion stack: 
- PagedAttention.write_to_paged_cache
- _custom_ops.reshape_and_cache
- torch.ops._C_cache_ops.reshape_and_cache
- cache_kernels.cu/reshape_and_cache
- CALL_RESHAPE_AND_CACHE (macro)
- reshape_and_cache_kernel
- fp8::scaled_convert
- scaled_vec_conversion

In [ ]:
### test 1: No prefix caching, fp8 quantization, use v1 block manager, no cuda graphs ###
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "fp8"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = False
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16",
                         max_model_len=1024*1, 
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2, 
                         debug_kv_sharing=DEBUG_KV_SHARING)

INFO 10-11 07:32:25 config.py:629] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. Meanwhile, it may cause accuracy drop without a proper scaling factor
INFO 10-11 07:32:25 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=fp8, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_trac

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-11 07:32:27 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.34it/s]



INFO 10-11 07:32:28 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-11 07:32:29 gpu_executor.py:122] # GPU blocks: 16610, # CPU blocks: 131072
INFO 10-11 07:32:32 model_runner.py:1383] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-11 07:32:32 model_runner.py:1387] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-11 07:33:14 model_runner.py:1511] Graph capturing finished in 43 secs.


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.allclose(attn_output_layer0, attn_output_layer1, atol=1e-2, rtol=1e-2)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 186.49it/s, est. speed input: 566.25 toks/s, output: 188.50 toks/s]


In [ ]:
model.model.layers[0].self_attn.attn_outputs, model.model.layers[1].self_attn.attn_outputs

([tensor([[ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
          [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114],
          [ 0.0029,  0.0021,  0.0306,  ...,  0.0140, -0.0356,  0.0200],
          [ 0.0019,  0.0014,  0.0311,  ...,  0.0135, -0.0352,  0.0198],
          [ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
          [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114]],
         device='cuda:0', dtype=torch.bfloat16)],
 [tensor([[ 0.0117,  0.0000,  0.0293,  ...,  0.0000, -0.0020,  0.0000],
          [ 0.0019,  0.0072,  0.0339,  ...,  0.0559, -0.0337,  0.0112],
          [ 0.0035,  0.0025,  0.0310,  ...,  0.0140, -0.0369,  0.0195],
          [ 0.0025,  0.0018,  0.0315,  ...,  0.0135, -0.0364,  0.0193],
          [ 0.0117,  0.0000,  0.0293,  ...,  0.0000, -0.0020,  0.0000],
          [ 0.0019,  0.0072,  0.0339,  ...,  0.0559, -0.0337,  0.0112]],
         device='cuda:0', dtype=torch.bfloat16)])

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
# output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

output = test_llm_generator.generate(78*["hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))


# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.allclose(attn_output_layer0, attn_output_layer1, atol=1e-2, rtol=1e-2)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 2452.92it/s, est. speed input: 4920.23 toks/s, output: 2459.56 toks/s]


In [ ]:
torch.allclose(torch.cat(model.model.layers[0].self_attn.attn_outputs), 
               torch.cat(model.model.layers[1].self_attn.attn_outputs), atol=1e-2, rtol=1e-2)

True

In [ ]:
torch.stack(model.model.layers[1].self_attn.attn_outputs).shape

torch.Size([1, 156, 4096])

In [ ]:
torch.stack(model.model.layers[0].self_attn.attn_outputs)

tensor([[[ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0337,  ...,  0.0557, -0.0334,  0.0114],
         [ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         ...,
         [ 0.0013,  0.0067,  0.0337,  ...,  0.0557, -0.0334,  0.0114],
         [ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0337,  ...,  0.0557, -0.0334,  0.0114]]],
       device='cuda:0', dtype=torch.bfloat16)

In [ ]:
torch.stack(model.model.layers[1].self_attn.attn_outputs)

tensor([[[ 0.0117,  0.0000,  0.0293,  ...,  0.0000, -0.0020,  0.0000],
         [ 0.0020,  0.0071,  0.0339,  ...,  0.0559, -0.0337,  0.0112],
         [ 0.0117,  0.0000,  0.0293,  ...,  0.0000, -0.0020,  0.0000],
         ...,
         [ 0.0020,  0.0071,  0.0339,  ...,  0.0559, -0.0337,  0.0112],
         [ 0.0117,  0.0000,  0.0293,  ...,  0.0000, -0.0020,  0.0000],
         [ 0.0020,  0.0071,  0.0339,  ...,  0.0559, -0.0337,  0.0112]]],
       device='cuda:0', dtype=torch.bfloat16)

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
# output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

output = test_llm_generator.generate(78*["hello"], 
                                     SamplingParams(temperature=0.0, ignore_eos=True, max_tokens=3))


# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.allclose(attn_output_layer0, attn_output_layer1, atol=1e-2, rtol=1e-2)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 1738.74it/s, est. speed input: 3484.94 toks/s, output: 5226.57 toks/s]


In [ ]:
reset_attn_outputs_and_metadatas(model)
model.model.attn_metadatas

[]

In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.allclose(attn_output_layer0, attn_output_layer1, atol=1e-2, rtol=1e-2)

# 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
assert len(model.model.attn_metadatas) == 3 
# Prefill phase.
attn_metadata = model.model.attn_metadatas[0]
assert attn_metadata.decode_metadata is None
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# Decode phase 1.
attn_metadata = model.model.attn_metadatas[1]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# Decode phase 2.
attn_metadata = model.model.attn_metadatas[2]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 157.38it/s, est. speed input: 475.34 toks/s, output: 475.05 toks/s]


### TEST 3 (cuda graphs)

In [ ]:
### test 1: No prefix caching, bf16 quantization, use v1 block manager, cuda graph mode ###
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = False
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16", 
                         max_model_len=1024*1, 
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

INFO 10-10 15:33:06 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/k/models/dummy-llama, use_v2_block_manager=True, num_scheduler_steps=1, chunked_prefill_

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-10 15:33:08 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.24it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.24it/s]



INFO 10-10 15:33:09 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-10 15:33:09 gpu_executor.py:122] # GPU blocks: 8049, # CPU blocks: 65536
INFO 10-10 15:33:09 model_runner.py:1383] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-10 15:33:09 model_runner.py:1387] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-10 15:33:39 model_runner.py:1511] Graph capturing finished in 30 secs.


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 227.94it/s, est. speed input: 690.65 toks/s, output: 230.00 toks/s]


In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.equal(attn_output_layer0, attn_output_layer1)



# XFormersCLAMetadata
#     NOTE: Any python object stored here is not updated when it is
#     cuda-graph replayed. If you have values that need to be changed
#     dynamically, it should be stored in tensor. The tensor has to be
#     updated from `CUDAGraphRunner.forward` API.
    
    
# # 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
# assert len(model.model.attn_metadatas) == 3 
# # Prefill phase.
# attn_metadata = model.model.attn_metadatas[0]
# assert attn_metadata.decode_metadata is None
# expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
# assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# # Decode phase 1.
# attn_metadata = model.model.attn_metadatas[1]
# assert attn_metadata.prefill_metadata is None
# expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
# assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# # Decode phase 2.
# attn_metadata = model.model.attn_metadatas[2]
# assert attn_metadata.prefill_metadata is None
# expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
# assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 102.69it/s, est. speed input: 310.07 toks/s, output: 309.86 toks/s]


### TEST 4 (block manager v1)

In [ ]:
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = False
ENFORCE_EAGER = True
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16", 
                         max_model_len=1024*1,
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

WARNING 10-10 15:33:53 config.py:365] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-10 15:33:53 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forw

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-10 15:33:55 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.31it/s]



INFO 10-10 15:33:56 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-10 15:33:56 gpu_executor.py:122] # GPU blocks: 8049, # CPU blocks: 65536


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 206.04it/s, est. speed input: 637.58 toks/s, output: 212.05 toks/s]


In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.equal(attn_output_layer0, attn_output_layer1)

# 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
assert len(model.model.attn_metadatas) == 3 
# Prefill phase.
attn_metadata = model.model.attn_metadatas[0]
assert attn_metadata.decode_metadata is None
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# Decode phase 1.
attn_metadata = model.model.attn_metadatas[1]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# Decode phase 2.
attn_metadata = model.model.attn_metadatas[2]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 110.57it/s, est. speed input: 333.76 toks/s, output: 333.53 toks/s]


### TEST 5 (prefix caching)

In [ ]:
from vllm.core.block.interfaces import Device

In [ ]:
# enforce xformers attention backend - changes are made here
os.environ["VLLM_ATTENTION_BACKEND"] = "XFORMERS_CLA"

In [ ]:
### test 1: No prefix caching, bf16 quantization, use v1 block manager, cuda graph mode ###
ENABLE_PREFIX_CACHING = True
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = False
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1,
                         dtype="bfloat16",
                         max_model_len=1024,
                         kv_cache_map={0:0,1:0},
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

INFO 10-11 12:09:14 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/k/models/dummy-llama, use_v2_block_manager=True, num_scheduler_steps=1, chunked_prefill_

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-11 12:09:15 selector.py:122] Using XFormers CLA backend.
INFO 10-11 12:09:16 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...
INFO 10-11 12:09:16 selector.py:122] Using XFormers CLA backend.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.33it/s]



INFO 10-11 12:09:17 model_runner.py:1060] Loading model weights took 2.8008 GB
INFO 10-11 12:09:17 gpu_executor.py:122] # GPU blocks: 5937, # CPU blocks: 65536
INFO 10-11 12:09:20 model_runner.py:1383] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-11 12:09:20 model_runner.py:1387] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-11 12:09:35 model_runner.py:1511] Graph capturing finished in 15 secs.


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
scheduler = test_llm_generator.llm_engine.scheduler[0]
gpu_allocator = scheduler.block_manager.block_allocator._allocators[Device.GPU]
cpu_allocator = scheduler.block_manager.block_allocator._allocators[Device.CPU]
gpu_allocator._cached_blocks, cpu_allocator._cached_blocks

({}, {})

In [ ]:
# scheduler.block_manager.gpu_allocator.cached_blocks, scheduler.block_manager.cpu_allocator.cached_blocks

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
# Just prefill with prefix cached.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], 
                                     SamplingParams(temperature=0.0, max_tokens=3))

Processed prompts: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 35.54it/s, est. speed input: 106.86 toks/s, output: 106.85 toks/s]


In [ ]:
for meta in model.model.attn_metadatas:
    if meta.prefill_metadata: print(meta.prefill_metadata.shared_self_attention_types)
    if meta.decode_metadata: print(meta.decode_metadata.shared_self_attention_types)

['PREFILL_KV_NEW', 'PREFILL_KV_SHARED']


In [ ]:
assert len(model.model.attn_metadatas) == 1

In [ ]:
scheduler.block_manager.get_prefix_cache_hit_rate(Device.GPU)

0.0

In [ ]:
examples = ["vLLM is a high-throughput and memory-efficient inference and serving engine for LLMs.",
            "Briefly describe the major milestones in the development of artificial intelligence from 1950 to 2020.",
            "Compare and contrast artificial intelligence with human intelligence in terms of processing information.",
            "Describe the basic components of a neural network and how it can be trained.",
            "Write a short story about a robot that dreams for the first time.",
            "Analyze the impact of the COVID-19 pandemic on global economic structures and future business models.",
            "Explain the cultural significance of the Mona Lisa painting, and how its perception might vary in Western versus Eastern societies.",
            "Translate the following English sentence into Japanese, French, and Swahili: 'The early bird catches the worm.'"]

In [ ]:
# Prefill and decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate([examples[0]], 
                                     SamplingParams(temperature=0.0, max_tokens=3))

Processed prompts: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 40.33it/s, est. speed input: 850.30 toks/s, output: 121.40 toks/s]


In [ ]:
output[0].outputs[0].text

"'gcfds copp"

In [ ]:
for meta in model.model.attn_metadatas:
    if meta.prefill_metadata: print(meta.prefill_metadata.shared_self_attention_types)
    if meta.decode_metadata: print(meta.decode_metadata.shared_self_attention_types)

['PREFILL_KV_NEW', 'PREFILL_KV_SHARED']


In [ ]:
assert torch.equal(torch.cat(model.model.layers[0].self_attn.attn_outputs),
                   torch.cat(model.model.layers[1].self_attn.attn_outputs))

In [ ]:
# Prefill and decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(examples, 
                                     SamplingParams(temperature=0.0, max_tokens=3))

Processed prompts: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 19.91it/s, est. speed input: 390.89 toks/s, output: 59.75 toks/s]


In [ ]:
output[0].outputs[0].text

"'gcfds copp"

In [ ]:
assert len(model.model.attn_metadatas) == 3

AssertionError: 

In [ ]:
for meta in model.model.attn_metadatas:
    if meta.prefill_metadata: print(meta.prefill_metadata.shared_self_attention_types)
    if meta.decode_metadata: print(meta.decode_metadata.shared_self_attention_types)

['PREFILL_PREFIX_CACHED_KV', 'PREFILL_PREFIX_CACHED_KV_SHARED']


In [ ]:
assert torch.equal(torch.cat(model.model.layers[0].self_attn.attn_outputs),
                   torch.cat(model.model.layers[1].self_attn.attn_outputs))

In [ ]:
model.model.layers[0].self_attn.attn_outputs[0]

tensor([[ 0.0136,  0.0037,  0.0108,  ...,  0.0039,  0.0063,  0.0105],
        [ 0.0128,  0.0038,  0.0075,  ...,  0.0041,  0.0063,  0.0104],
        [ 0.0119,  0.0038,  0.0067,  ...,  0.0029,  0.0049,  0.0079],
        ...,
        [ 0.0042,  0.0115, -0.0127,  ...,  0.0089,  0.0091, -0.0011],
        [ 0.0041,  0.0115, -0.0128,  ...,  0.0086,  0.0092, -0.0009],
        [ 0.0039,  0.0112, -0.0125,  ...,  0.0082,  0.0089, -0.0015]],
       device='cuda:0', dtype=torch.bfloat16)

In [ ]:
model.model.layers[1].self_attn.attn_outputs[0]

tensor([[ 0.0136,  0.0037,  0.0108,  ...,  0.0039,  0.0063,  0.0105],
        [ 0.0128,  0.0038,  0.0075,  ...,  0.0041,  0.0063,  0.0104],
        [ 0.0119,  0.0038,  0.0067,  ...,  0.0029,  0.0049,  0.0079],
        ...,
        [ 0.0042,  0.0115, -0.0127,  ...,  0.0089,  0.0091, -0.0011],
        [ 0.0041,  0.0115, -0.0128,  ...,  0.0086,  0.0092, -0.0009],
        [ 0.0039,  0.0112, -0.0125,  ...,  0.0082,  0.0089, -0.0015]],
       device='cuda:0', dtype=torch.bfloat16)

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 213.26it/s, est. speed input: 650.58 toks/s, output: 216.44 toks/s]


AssertionError: 

In [ ]:
assert scheduler.block_manager.get_prefix_cache_hit_rate(Device.GPU) == 0.0

In [ ]:
# Just prefill with prefix cached.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate("hello world!", SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 59.95it/s, est. speed input: 241.74 toks/s, output: 181.08 toks/s]


In [ ]:
# Just prefill with prefix cached.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate("hey", SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 57.29it/s, est. speed input: 115.55 toks/s, output: 173.07 toks/s]


In [ ]:
scheduler.block_manager.gpu_allocator.cached_blocks, scheduler.block_manager.cpu_allocator.cached_blocks

AttributeError: 'BlockSpaceManagerV2' object has no attribute 'gpu_allocator'

In [ ]:
scheduler.block_manager.gpu_allocator

AttributeError: 'BlockSpaceManagerV2' object has no attribute 'gpu_allocator'

In [ ]:
scheduler.block_manager.gpu_allocator.cache_metric_data.block_size

1000

In [ ]:
scheduler.block_manager.gpu_allocator.cache_metric_data

CacheMetricData(num_completed_blocks=0, completed_block_cache_hit_rate=0.0, num_incompleted_block_queries=17, num_incompleted_block_hit=14, block_size=1000)

In [ ]:
scheduler.block_manager.gpu_allocator.cache_metric_data.num_completed_blocks

0

In [ ]:
scheduler.block_manager.get_prefix_cache_hit_rate(Device.GPU)

0.8235294117647058

In [ ]:
scheduler.block_manager.gpu_allocator.cache_metric_data.num_incompleted_block_hit

14

In [ ]:
scheduler.block_manager.gpu_allocator.cache_metric_data.num_incompleted_block_queries

17